In [1]:
import pandas as pd
from datetime import date


In [2]:
today = date.today()
today_str = today.strftime('%Y%m%d')
print(today_str)

20180521


In [3]:
df = pd.read_hdf('X:/Applications/Admissions/funnel/data/stage_data', key='weekly')
df = df[(df['year_term']>'2011.Spring')]


In [4]:
summ = df.groupby(['year_term', 'stage']).sum()
summ_t = summ.transpose()


In [5]:
print(summ.shape)
summ.head()

(42, 54)


00  01  02  03  04  05  06  07  08  09 ...    44   45  \
year_term   stage                                             ...              
2012.Fall   Accepted    1   1   1   1   1   1   1   2   2   2 ...   401  398   
            Applied     1   1   1   2   5   6   6   9  11  21 ...   554  552   
            Deposited   3   3   3   3   3   3   3   4   4   4 ...   366  366   
2012.Spring Accepted    0   0   0   0   0   0   0   0   0   1 ...     1    1   
            Applied     0   0   0   0   1   1   1   1   1   1 ...     2    2   

                        46   47   48   49   50   51   52   53  
year_term   stage                                              
2012.Fall   Accepted   381  380  344  345  328  275  261  261  
            Applied    531  526  478  479  462  405  392  392  
            Deposited  367  367  370  370  371  363  363  363  
2012.Spring Accepted     1    1    1    1    1    1    1    1  
            Applied      2    2    2    2    2    2    2    2  

[5 rows x 54 columns]

In [6]:
summ_t.head()

year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                1       1         3           0       0         0        16   
01                1       1         3           0       0         0        18   
02                1       1         3           0       0         0        21   
03                1       2         3           0       0         0        22   
04                1       5         3           0       1         0        25   

year_term                   2013.Spring    ...    2017.Fall 2017.Spring  \
stage     Applied Deposited    Accepted    ...    Deposited    Accepted   
00             82         7           8    ...           10           8   
01             72         7           8    ...           10           8   
02             76         8           8    ...           11           8   
03             92        11          13    ...           11           8   
04            104        11          14    ...           11          10   

year_term                   2018.Fall                   2018.Spring          \
stage     Applied Deposited  Accepted Applied Deposited    Accepted Applied   
00             12         4         8      27         3           2       4   
01             12         4         9      33         3           2       4   
02             12         4         9      35         3           2       5   
03             13         4         9      43         3           2       6   
04             14         4         9      51         3           2       8   

year_term            
stage     Deposited  
00                3  
01                3  
02                3  
03                3  
04                3  

[5 rows x 42 columns]

In [ ]:
summ.loc[(summ['year_term'] == '2017.Fall') & (summ['stage'] == 'Deposited')]

In [7]:
print(summ.loc[('2018.Fall', 'Deposited')]['38'])
print(summ.loc[('2017.Fall', 'Deposited')]['38'])

summ.loc[('2018.Fall', 'Deposited')]['38'] - summ.loc[('2017.Fall', 'Deposited')]['38']

191
201


-10

In [8]:
print(summ.loc[('2017.Fall', 'Deposited')]['53'])
print(summ.loc[('2017.Fall', 'Deposited')]['38'])

summ.loc[('2017.Fall', 'Deposited')]['53'] - summ.loc[('2017.Fall', 'Deposited')]['38']

259
201


58

In [9]:
weeks = range(0, 54)
for w in weeks:
    f = f'{w:02d}'
    yet_to_deposit = summ.loc[('2017.Fall', 'Deposited')]['53'] - summ.loc[('2017.Fall', 'Deposited')][f]
    print(w,yet_to_deposit)

0 249
1 249
2 248
3 248
4 248
5 248
6 248
7 248
8 248
9 248
10 246
11 246
12 246
13 243
14 238
15 235
16 228
17 227
18 227
19 220
20 216
21 212
22 209
23 203
24 202
25 196
26 181
27 175
28 173
29 167
30 159
31 148
32 140
33 132
34 113
35 88
36 69
37 62
38 58
39 48
40 40
41 34
42 31
43 25
44 18
45 14
46 10
47 9
48 -2
49 -9
50 2
51 2
52 0
53 0


In [10]:
summ.describe()

,00,01,02,03,04,05,06,07,08,09,...,44,45,46,47,48,49,50,51,52,53
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,...,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,17.833333,18.380952,20.309524,23.000000,25.404762,28.738095,33.428571,38.738095,47.000000,56.690476,...,284.976190,282.309524,280.333333,278.880952,272.452381,261.071429,249.880952,243.785714,241.023810,241.023810
std,23.306983,23.524717,26.469023,30.250983,33.336466,37.955725,45.150233,53.672930,64.244958,78.460333,...,319.321345,316.380019,315.136646,313.720258,307.844687,293.509994,283.305678,280.611052,278.963193,278.963193
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.250000,3.250000,4.250000,5.250000,5.250000,6.250000,...,28.250000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
50%,8.000000,8.500000,8.500000,11.000000,11.000000,11.000000,13.500000,15.500000,16.500000,19.500000,...,119.500000,119.500000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
75%,23.000000,23.750000,25.500000,30.000000,31.000000,37.250000,41.000000,50.750000,70.250000,76.750000,...,435.500000,419.000000,409.500000,395.750000,372.250000,354.250000,344.250000,332.500000,324.250000,324.250000
max,90.000000,89.000000,103.000000,124.000000,132.000000,146.000000,174.000000,210.000000,245.000000,296.000000,...,1024.000000,992.000000,984.000000,974.000000,937.000000,936.000000,924.000000,929.000000,925.000000,925.000000


In [11]:
summ.index

MultiIndex(levels=[['2012.Fall', '2012.Spring', '2013.Fall', '2013.Spring', '2014.Fall', '2014.Spring', '2015.Fall', '2015.Spring', '2016.Fall', '2016.Spring', '2017.Fall', '2017.Spring', '2018.Fall', '2018.Spring'], ['Accepted', 'Applied', 'Deposited']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['year_term', 'stage'])

In [12]:
#Goal 
GOAL = 260
summ.loc[('2018.Fall', 'Deposited')]['53'] = GOAL

In [13]:
df1 = summ.copy()
weeks = range(0, 54)
for w in weeks:
    f = f'{w:02d}'
    df1[f] = (summ.loc[:]['53'] - summ.loc[:][f])

print(df1.shape)

(42, 54)


In [14]:
df1.head()

00   01   02   03   04   05   06   07   08   09 ...  \
year_term   stage                                                       ...   
2012.Fall   Accepted   260  260  260  260  260  260  260  259  259  259 ...   
            Applied    391  391  391  390  387  386  386  383  381  371 ...   
            Deposited  360  360  360  360  360  360  360  359  359  359 ...   
2012.Spring Accepted     1    1    1    1    1    1    1    1    1    0 ...   
            Applied      2    2    2    2    1    1    1    1    1    1 ...   

                        44   45   46   47  48  49  50  51  52  53  
year_term   stage                                                  
2012.Fall   Accepted  -140 -137 -120 -119 -83 -84 -67 -14   0   0  
            Applied   -162 -160 -139 -134 -86 -87 -70 -13   0   0  
            Deposited   -3   -3   -4   -4  -7  -7  -8   0   0   0  
2012.Spring Accepted     0    0    0    0   0   0   0   0   0   0  
            Applied      0    0    0    0   0   0   0   0   0   0  

[5 rows x 54 columns]

In [15]:
df1_t = df1.transpose()

In [16]:
df1_t.head()

year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00              260     391       360           1       2        39       281   
01              260     391       360           1       2        39       279   
02              260     391       360           1       2        39       276   
03              260     390       360           1       2        39       275   
04              260     387       360           1       1        39       272   

year_term                   2013.Spring    ...    2017.Fall 2017.Spring  \
stage     Applied Deposited    Accepted    ...    Deposited    Accepted   
00            230       292          21    ...          249           8   
01            240       292          21    ...          249           8   
02            236       291          21    ...          248           8   
03            220       288          16    ...          248           8   
04            208       288          15    ...          248           6   

year_term                   2018.Fall                   2018.Spring          \
stage     Applied Deposited  Accepted Applied Deposited    Accepted Applied   
00              6        10       818     817       257          22      24   
01              6        10       817     811       257          22      24   
02              6        10       817     809       257          22      23   
03              5        10       817     801       257          22      22   
04              4        10       817     793       257          22      20   

year_term            
stage     Deposited  
00               19  
01               19  
02               19  
03               19  
04               19  

[5 rows x 42 columns]

In [17]:
#from bokeh.layouts import widgetbox, row
#from bokeh.models.widgets import MultiSelect, RadioGroup, Select
#from bokeh.plotting import figure, curdoc
from bokeh.plotting import figure, output_file, show
#from bokeh.palettes import Blues9
from bokeh.palettes import Set2_8
from bokeh.io import output_notebook


In [18]:
output_notebook()
output_file("DepositsToEndOFYear.html")

Loading BokehJS ...

In [19]:
TOOLS="pan,wheel_zoom,box_zoom,save,reset"

In [20]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1
    
    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = week_number - adm_start_week
    else:
        adm_week_number = 53 + (week_number - adm_start_week)
    
    return (week_number, adm_week_number)

week_number, adm_week_number = adm_week(today)


In [21]:
stage = 'Deposited'
term = '2018.Fall'

all_terms = sorted(list(df['year_term'].dropna().unique()))
all_terms = [l for l in all_terms if 'Fall' in l]
all_terms.remove(term)
#all_terms.remove('2013.Fall')
all_terms.remove('2012.Fall')
all_terms.reverse()
y_max = df1_t[(term, stage)].max()
y_min = df1_t[(term, stage)].min()
for t in all_terms:
    ymx = df1_t[(t, stage)].max()
    ymn = df1_t[(t, stage)].min()
    if ymx > y_max:
        y_max = ymx
    if ymn < y_min:
        y_min = ymn

print(term, stage, y_min, y_max)

2018.Fall Deposited -15 317


In [22]:
print(all_terms)

['2017.Fall', '2016.Fall', '2015.Fall', '2014.Fall', '2013.Fall']


In [23]:
title = f"'{stage}' Gained to End of Year,  Goal={GOAL}  (Week {adm_week_number:d})"

In [24]:
p = figure(plot_width=800, plot_height=600, title=title,
       x_axis_label="Admissions Week Number (year starts Sept 1)",
       y_axis_label=stage,
       tools=TOOLS,
       x_range=(0,54),
       y_range=(y_min*1.05, y_max*1.05)
      )

c=0
for t in all_terms:
    #p.line(df1_t.index, df1_t[(t, stage)], color=Blues9[c], legend=t)
    p.line(df1_t.index, df1_t[(t, stage)], color=Set2_8[c], legend=t)
    if c <= 7:
        c += 1

p.line(df1_t.index, df1_t[(term, stage)], 
       color='red', line_width=2, legend=term)

# week_number line
p.line((adm_week_number,adm_week_number),
       (-1000,5000), color='green', line_width=0.8, line_dash='dashed',
       legend=f'Week {adm_week_number:d}', alpha=0.8)

p.legend.location = "top_right"

show(p)